### Oniel Gracious - Competition 3 - Santander

In [71]:
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier,Pool
from sklearn.linear_model import LinearRegression
from IPython.display import display

import matplotlib.patches as patch
import matplotlib.pyplot as plt
from sklearn.svm import NuSVR
from scipy.stats import norm
from sklearn import svm
import xgboost as xgb
import seaborn as sns
import glob
import sys

import numpy as np
import pandas as pd
import os
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')
import gc
import time
plt.style.use('seaborn')
sns.set(font_scale=1)
from tqdm import tnrange, tqdm, tqdm_notebook

In [4]:
train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')

In [6]:
train.shape,test.shape

((200000, 202), (200000, 201))

In [7]:
# Identify real ids and fake ID's from test

In [ ]:
features = [c for c in train.columns if c not in ['ID_code']]

In [8]:
# Brilliant discovery for this competition, this was the secret to the magic variable. I didnt find it.
df_test = test[features]
#df_test.drop(['ID_code'], axis=1, inplace=True)
df_test = df_test.values

unique_samples = []
unique_count = np.zeros_like(df_test)

for feature in tqdm(range(df_test.shape[1])):
    _, index_, count_ = np.unique(df_test[:, feature], return_counts=True, return_index=True)
    unique_count[index_[count_ == 1], feature] = 1

# Samples which have unique values are real the others are fake
real_samples_indexes = np.argwhere(np.sum(unique_count, axis=1) > 0)[:, 0]
synthetic_samples_indexes = np.argwhere(np.sum(unique_count, axis=1) == 0)[:, 0]

print(len(real_samples_indexes))
print(len(synthetic_samples_indexes))

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [00:06<00:00, 30.02it/s]


100000
100000


In [9]:
train.shape,test.shape

((200000, 202), (200000, 200))

In [11]:
test = pd.read_csv('input/test.csv')

In [12]:

fake_ids = list(synthetic_samples_indexes) #np.load('input/synthetic_samples_indexes.npy')

ids = np.arange(test.shape[0])

real_ids = list(set(ids) - set(fake_ids)) # NICE

real_test = test.iloc[real_ids]
fake_test = test.iloc[fake_ids]
real_test_id = real_test.ID_code

In [13]:
features = [c for c in train.columns if c not in ['target', 'ID_code']]

In [14]:
## Get the difference between Var and the Feature...now we have 400 variables

In [15]:
df = pd.concat([train,real_test], axis = 0)

for feat in tqdm_notebook(features):
    df[feat+'_var'] = df.groupby([feat])[feat].transform('var')
    
for feat in tqdm_notebook(features):
    df[feat+'plus_'] = df[feat] + df[feat+'_var']
    # df[feat+'minus_'] = df[feat] - df[feat+'_var'] this is not necessary

drop_features = [c for c in df.columns if '_var' in c]
df.drop(drop_features, axis=1, inplace=True)

In [16]:
train = df.iloc[:train.shape[0]]
real_test = df.iloc[train.shape[0]:]

In [22]:
# Add Row Level Features
%%time
idx = features = train_df.columns.values[2:202]

#for df in [train, real_test]:
train['sum'] = train[idx].sum(axis=1)  
train['min'] = train[idx].min(axis=1)
train['max'] = train[idx].max(axis=1)
train['mean'] = train[idx].mean(axis=1)
train['std'] = train[idx].std(axis=1)
train['skew'] = train[idx].skew(axis=1)
train['kurt'] = train[idx].kurtosis(axis=1)
train['med'] = train[idx].median(axis=1)
    
#for df in [train, real_test]:
real_test['sum'] = real_test[idx].sum(axis=1)  
real_test['min'] = real_test[idx].min(axis=1)
real_test['max'] = real_test[idx].max(axis=1)
real_test['mean'] = real_test[idx].mean(axis=1)
real_test['std'] = real_test[idx].std(axis=1)
real_test['skew'] = real_test[idx].skew(axis=1)
real_test['kurt'] = real_test[idx].kurtosis(axis=1)
real_test['med'] = real_test[idx].median(axis=1)

UsageError: Line magic function `%%time` not found.


In [23]:
# Frequency Encode
def frequency_encoding(variable):
    t = pd.concat([train[variable], real_test[variable]]).value_counts().reset_index()
    t = t.reset_index()
    t.loc[t[variable] == 1, 'level_0'] = np.nan
    t.set_index('index', inplace=True)
    max_label = t['level_0'].max() + 1
    t.fillna(max_label, inplace=True)
    return t.to_dict()['level_0']

In [24]:
### Frequency Encoding - Didnt Work
#from tqdm import tqdm_notebook
#cols = []
#cols = [c for c in train.columns if c not in ['ID_code', 'target']]

##for df in [train, real_test]:
#for col in tqdm_notebook(cols):
#    freq_enc_dict = frequency_encoding(col)
#    train['encode_FE' + col] = train[col].map(lambda x: freq_enc_dict.get(x, np.nan))
#    real_test['encode_FE' + col] = real_test[col].map(lambda x: freq_enc_dict.get(x, np.nan))
        

In [36]:
features = [c for c in train.columns if c not in ['ID_code', 'target']]
target = train['target'] 

In [26]:
print(train.shape)
print(real_test.shape)
print(len(features))

(200000, 802)
(100000, 802)
800


In [27]:
param = {
    'bagging_freq': 5,  'bagging_fraction': 0.4,  'boost_from_average':'false',   
    'boost': 'gbdt',    'feature_fraction': 0.04, 'learning_rate': 0.01,
    'max_depth': -1,    'metric':'auc',             'min_data_in_leaf': 80,
    'num_leaves': 13,  'num_threads': 8,            
    'tree_learner': 'serial',   'objective': 'binary',       'verbosity': 1
}

In [28]:
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=4242)
oof = np.zeros(len(train))
predictions = np.zeros(len(real_test))
val_aucs = []
feature_importance_df = pd.DataFrame()

In [29]:
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train.values, target.values)):
    print("Fold {}".format(fold_))
    trn_data = lgb.Dataset(train.iloc[trn_idx][features], label=target.iloc[trn_idx])
    val_data = lgb.Dataset(train.iloc[val_idx][features], label=target.iloc[val_idx])
    
    clf = lgb.train(param, trn_data, 100000, valid_sets = [trn_data, val_data], verbose_eval=5000, early_stopping_rounds = 3000)
    
    oof[val_idx] = clf.predict(train.iloc[val_idx][features], num_iteration=clf.best_iteration)
    val_aucs.append(roc_auc_score(target[val_idx] , oof[val_idx] ))
    
    predictions += clf.predict(real_test[features], num_iteration=clf.best_iteration) / folds.n_splits
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = features
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

Fold 0
Training until validation scores don't improve for 3000 rounds.
[5000]	training's auc: 0.95253	valid_1's auc: 0.92293
[10000]	training's auc: 0.974282	valid_1's auc: 0.924532
Early stopping, best iteration is:
[10817]	training's auc: 0.976889	valid_1's auc: 0.92458
Fold 1
Training until validation scores don't improve for 3000 rounds.
[5000]	training's auc: 0.952809	valid_1's auc: 0.921332
[10000]	training's auc: 0.974394	valid_1's auc: 0.92303
Early stopping, best iteration is:
[10021]	training's auc: 0.974462	valid_1's auc: 0.923043
Fold 2
Training until validation scores don't improve for 3000 rounds.
[5000]	training's auc: 0.953069	valid_1's auc: 0.917857
[10000]	training's auc: 0.974759	valid_1's auc: 0.919899
Early stopping, best iteration is:
[9782]	training's auc: 0.974009	valid_1's auc: 0.919972
Fold 3
Training until validation scores don't improve for 3000 rounds.
[5000]	training's auc: 0.953068	valid_1's auc: 0.920307
[10000]	training's auc: 0.974559	valid_1's auc: 0.

In [30]:
mean_auc = np.mean(val_aucs)
std_auc = np.std(val_aucs)
all_auc = roc_auc_score(target, oof)
print("Mean auc: %.9f, std: %.9f. All auc: %.9f." % (mean_auc, std_auc, all_auc))

Mean auc: 0.921913832, std: 0.001935783. All auc: 0.921809452.


In [31]:
subreal = pd.DataFrame({"ID_code": real_test_id.values})
subreal['target']=predictions
sub = pd.DataFrame({"ID_code": test.ID_code.values})

In [32]:
finalsub = sub.set_index('ID_code').join(subreal.set_index('ID_code')).reset_index()
finalsub.fillna(0,inplace=True)
finalsub.to_csv("submission-Late1.csv", index=False)

In [38]:
from sklearn.cross_validation import train_test_split
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.tree import DecisionTreeClassifier

train_X, val_X, train_y, val_y = train_test_split(train, target, random_state=1)


In [40]:
train_X.head()

,ID_code,target,var_0,var_1,var_10,var_100,var_101,var_102,var_103,var_104,...,encode_FEvar_190plus_,encode_FEvar_191plus_,encode_FEvar_192plus_,encode_FEvar_193plus_,encode_FEvar_194plus_,encode_FEvar_195plus_,encode_FEvar_196plus_,encode_FEvar_197plus_,encode_FEvar_198plus_,encode_FEvar_199plus_
76034,train_76034,0.0,10.3756,-1.3213,0.5893,-15.4467,5.4248,21.2245,1.6883,9.7037,...,2197.0,13266.0,NaN,25496.0,NaN,2021.0,NaN,16348.0,30478.0,NaN
51652,train_51652,0.0,7.3689,-3.9301,-4.7831,19.5275,20.0585,21.5887,1.8115,11.1366,...,17712.0,57889.0,15427.0,5559.0,NaN,15810.0,NaN,6549.0,30353.0,24905.0
138222,train_138222,1.0,8.0033,-8.8002,7.3013,-8.1361,24.3871,31.1576,1.4519,9.1671,...,NaN,15330.0,27878.0,61579.0,69644.0,10058.0,12363.0,1133.0,NaN,40181.0
115575,train_115575,0.0,13.3624,0.8846,-6.2352,-13.2246,11.4086,12.5656,1.5613,12.4782,...,65519.0,5683.0,51088.0,40269.0,11829.0,9252.0,43372.0,11306.0,NaN,NaN
45249,train_45249,0.0,9.3283,-7.8924,12.3823,0.4774,10.3129,36.7297,1.6154,6.9304,...,NaN,9276.0,4328.0,NaN,NaN,841.0,46093.0,204.0,39104.0,37229.0


In [41]:
# Cat
train_pool = Pool(train_X[features],train_y)
cat_model = CatBoostClassifier(
                               iterations=3000,# change 25 to 3000 to get best performance 
                               learning_rate=0.03,
                               objective="Logloss",
                               eval_metric='AUC',
                              )
cat_model.fit(train_X[features],train_y,silent=True)


In [56]:
train_X = train_X.replace(np.nan, 0)
val_X = val_X.replace(np.nan, 0)

train = train.replace(np.nan, 0)
real_test = real_test.replace(np.nan, 0)

train_X[features].shape,train_X[features].isnull().sum().sum()
#,train_X[features].isnan().sum()


((150000, 800), 0)

In [57]:
train_X[features].isnull().sum()

var_0                    0
var_1                    0
var_10                   0
var_100                  0
var_101                  0
var_102                  0
var_103                  0
var_104                  0
var_105                  0
var_106                  0
var_107                  0
var_108                  0
var_109                  0
var_11                   0
var_110                  0
var_111                  0
var_112                  0
var_113                  0
var_114                  0
var_115                  0
var_116                  0
var_117                  0
var_118                  0
var_119                  0
var_12                   0
var_120                  0
var_121                  0
var_122                  0
var_123                  0
var_124                  0
                        ..
encode_FEvar_170plus_    0
encode_FEvar_171plus_    0
encode_FEvar_172plus_    0
encode_FEvar_173plus_    0
encode_FEvar_174plus_    0
encode_FEvar_175plus_    0
e

In [58]:
# Random Forest
rfc_model = RandomForestClassifier(random_state=0).fit(train_X[features], train_y)

In [59]:
# Decision Tree
tree_model = DecisionTreeClassifier(random_state=0, max_depth=5, min_samples_split=5).fit(train_X[features], train_y)

In [61]:
# Prediction Models
#y_pred_lgb_train = lgb.predict(train[features])
y_pred_cat_train = cat_model.predict(train[features])
y_pred_rfc_train = rfc_model.predict(train[features])
y_pred_tree_train = tree_model.predict(train[features])

In [62]:
# Score
# Cat Boost
mean_auc = np.mean(y_pred_cat_train)
std_auc = np.std(y_pred_cat_train)
all_auc = roc_auc_score(target, y_pred_cat_train)
print("Cat Boost Mean auc: %.9f, std: %.9f. All auc: %.9f." % (mean_auc, std_auc, all_auc))

# Random Forest
mean_auc = np.mean(y_pred_rfc_train)
std_auc = np.std(y_pred_rfc_train)
all_auc = roc_auc_score(target, y_pred_rfc_train)
print("Random Forest Mean auc: %.9f, std: %.9f. All auc: %.9f." % (mean_auc, std_auc, all_auc))

# Decision Tree
mean_auc = np.mean(y_pred_tree_train)
std_auc = np.std(y_pred_tree_train)
all_auc = roc_auc_score(target, y_pred_tree_train)
print("Decison Tree Mean auc: %.9f, std: %.9f. All auc: %.9f." % (mean_auc, std_auc, all_auc))

Cat Boost Mean auc: 0.053460000, std: 0.224948946. All auc: 0.734882055.
Random Forest Mean auc: 0.065680000, std: 0.247721896. All auc: 0.825249873.
Decison Tree Mean auc: 0.003675000, std: 0.060510283. All auc: 0.511177453.


In [64]:
# Predictions on Test
#y_pred_lgb_train = lgb.predict(X)
y_pred_cat = cat_model.predict(real_test[features])
y_pred_rfc = rfc_model.predict(real_test[features])
y_pred_tree = tree_model.predict(real_test[features])

In [65]:
# Ensemble and Stack

In [75]:
# Dataset that will be the train set of the ensemble model.
first_level = pd.DataFrame()
first_level['cat_boost'] = y_pred_cat_train
first_level['random_forest'] = y_pred_rfc_train
first_level['decision_tree'] = y_pred_tree_train
first_level['lgb'] = oof
#first_level['target'] = target.values
first_level.head(20)

,cat_boost,random_forest,decision_tree,lgb
0,0.0,0.0,0.0,0.001657
1,0.0,0.0,0.0,0.548527
2,0.0,0.0,0.0,0.005902
3,0.0,0.0,0.0,0.044770
4,0.0,0.0,0.0,0.040842
5,0.0,0.0,0.0,0.015414
6,0.0,0.0,0.0,0.161597
7,0.0,0.0,0.0,0.018159
8,0.0,0.0,0.0,0.093118
9,0.0,0.0,0.0,0.017863


In [76]:
# Dataset that will be the test set of the ensemble model.
first_level_test = pd.DataFrame()
first_level_test['cat_boost'] = y_pred_cat
first_level_test['random_forest'] = y_pred_rfc
first_level_test['decision_tree'] = y_pred_tree
first_level_test['lgb'] = predictions
first_level_test.head(20)

,cat_boost,random_forest,decision_tree,lgb
0,0.0,0.0,0.0,0.140687
1,0.0,0.0,0.0,0.067085
2,0.0,0.0,0.0,0.023961
3,0.0,0.0,0.0,0.025297
4,0.0,0.0,0.0,0.448054
5,0.0,0.0,0.0,0.020525
6,0.0,0.0,0.0,0.074498
7,1.0,1.0,0.0,0.559852
8,0.0,0.0,0.0,0.044138
9,0.0,0.0,0.0,0.009910


Ensemble architecture:

1st level:
Catboost
XGBM
Random forest
Linear Regression
KNN

2nd level;
Linear Regression

In [77]:
meta_model = LinearRegression(n_jobs=-1)

In [78]:
meta_model.fit(first_level, target)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False)

In [79]:
ensemble_pred = meta_model.predict(first_level)
final_predictions = meta_model.predict(first_level_test)

In [80]:
mean_auc = np.mean(ensemble_pred)
std_auc = np.std(ensemble_pred)
all_auc = roc_auc_score(target, ensemble_pred)
print("Cat Boost Mean auc: %.9f, std: %.9f. All auc: %.9f." % (mean_auc, std_auc, all_auc))

Cat Boost Mean auc: 0.100490000, std: 0.251728893. All auc: 0.969339222.


In [81]:
subreal = pd.DataFrame({"ID_code": real_test_id.values})
subreal['target']=final_predictions
sub = pd.DataFrame({"ID_code": test.ID_code.values})

In [82]:
finalsub = sub.set_index('ID_code').join(subreal.set_index('ID_code')).reset_index()
finalsub.fillna(0,inplace=True)
finalsub.to_csv("submission-Late2.csv", index=False)

In [83]:
## NICE
from IPython.display import HTML

def create_download_link(title = "Download CSV file", filename = "data.csv"):  
    html = '<a href={filename}>{title}</a>'
    html = html.format(title=title,filename=filename)
    return HTML(html)

# create a link to download the dataframe which was saved with .to_csv method
create_download_link(filename='submission-Late2.csv')